## Production of PDDL file from prompt

In [32]:
from llm_plus_p import llm_plus_p_function

query = 'Summarize high severity tickets from the customer UltimateCustomer'
pddl_code = llm_plus_p_function(query)

In [33]:
print(pddl_code)

(define (problem tool_problem)
  (:domain tool_domain)
  (:objects
    UltimateCustomer - object
    high - object
    ticket - object
  )
  (:init
    (tool_argument_value UltimateCustomer)
    (tool_argument_value high)
    (tool_argument_value ticket)
  )
  (:goal (and
    (tool_used summarize_objects)
    (argument_of summarize_objects UltimateCustomer high ticket)
  ))

  ;; Actions
  (:action use_tool_works_list
    :parameters ()
    :precondition (and)
    :effect (and
      (tool_used works_list)
      (argument_of works_list (ticket.severity high) (ticket.rev_org UltimateCustomer) (type ticket))
    )
  )

  (:action use_tool_summarize_objects
    :parameters ()
    :precondition (and
      (tool_used works_list)
      (argument_of works_list (ticket.severity high) (ticket.rev_org UltimateCustomer) (type ticket))
    )
    :effect (and
      (tool_used summarize_objects)
      (argument_of summarize_objects (objects $$PREV[1]))
    )
  )
)


In [38]:
import re

solution = []

encoded_tasks = pddl_code.split(':action')[1:]
for encoded_task in encoded_tasks:
     tasks = encoded_task.split(':effect')[1:]
     decoded_task = {}
     for task in tasks:
          data = task.split('argument_of')[1].split()
          decoded_task['tool_name'] = data[0]
          data = [re.sub('[()"]', '', i) for i in data][1:]
          decoded_task['arguments'] = []
          for i in range(0, len(data) - 1, 2):
               if data[i] == '': continue
               argument = {}
               argument['argument_name'] = data[i]
               argument['argument_value'] = data[i + 1]
               decoded_task['arguments'].append(argument)
          
     solution.append(decoded_task)

solution

[{'tool_name': 'works_list',
  'arguments': [{'argument_name': 'ticket.severity', 'argument_value': 'high'},
   {'argument_name': 'ticket.rev_org', 'argument_value': 'UltimateCustomer'},
   {'argument_name': 'type', 'argument_value': 'ticket'}]},
 {'tool_name': 'summarize_objects',
  'arguments': [{'argument_name': 'objects', 'argument_value': '$$PREV[1]'}]}]